# Smallworld Demo — Building a Clean, Validated Master Dataset from an SQLite Database

This notebook demonstrates a complete **data workflow** from raw relational tables in SQLite to a **curated, analysis-ready dataset**.

## What this demo does
1. **Loads** an SQLite database file
2. **Inspects** the available tables and previews the raw data
3. **Joins** the tables into a single master table using `Field_ID`
4. **Validates** the joined data (duplicates, missing values, basic range sanity)
5. **Curates** the dataset (clean schema + light standardization)
6. **Exports** the curated dataset to a CSV file for downstream analysis/modeling

## Expected output
- `smallworld_curated.csv` — the final curated dataset exported from `df_clean`


## 0) Loading the database

### Option A (Google Colab)
Upload the `.db` file when prompted (it will be placed in the Colab runtime folder).

### Option B (Local Jupyter)
Place the `.db` file in the same folder as this notebook (or set `DB_PATH` to its full path).


In [14]:
# If running on Colab, upload the SQLite DB here.
# If you're running locally, set DB_PATH manually.

try:
    from google.colab import files  # type: ignore
    uploaded = files.upload()
    DB_PATH = list(uploaded.keys())[0]  # selects the uploaded filename
    print("Uploaded DB:", DB_PATH)
except Exception:
    # Local / non-Colab: set your path here
    DB_PATH = "Smallworld_farm_survey_small.db"
    print("Using local DB_PATH:", DB_PATH)


Saving Smallworld_farm_survey_small.db to Smallworld_farm_survey_small.db
Uploaded DB: Smallworld_farm_survey_small.db


In [15]:
# Dependencies
# - In Colab, install SQLAlchemy if needed.
# - Locally, install from requirements.txt.

try:
    import sqlalchemy  # noqa: F401
except Exception:
    !pip -q install sqlalchemy


In [16]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{DB_PATH}")


## 1) Inspect the source tables

We list the tables contained in the SQLite database.  
This confirms we connected to the correct file and shows what raw sources are available.


In [17]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", engine)
tables


,name
0,geographic_features
1,weather_features
2,soil_and_crop_features
3,farm_management_features


## 2) Preview each table (raw data)

We preview a few rows per table to understand:
- column names and types
- whether keys like `Field_ID` exist consistently
- how each table contributes to the master dataset


In [18]:
from IPython.display import display

for t in tables["name"].tolist():
    print(f"\n=== {t} ===")
    display(pd.read_sql(f"SELECT * FROM {t} LIMIT 3;", engine))



=== geographic_features ===


,Field_ID,Elevation,Latitude,Longitude,Location,Slope
0,40734,786.0558,-7.389911,-7.556202,Rural_Akatsi,14.795113
1,30629,674.3341,-7.736849,-1.051539,Rural_Sokoto,11.374611
2,39924,826.5339,-9.926616,0.115156,Rural_Sokoto,11.339692



=== weather_features ===


,Field_ID,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps
0,40734,1125.2,-3.1,33.1,15.00
1,30629,1450.7,-3.9,30.6,13.35
2,39924,2208.9,-1.8,28.4,13.30



=== soil_and_crop_features ===


,Field_ID,Soil_fertility,Soil_type,pH
0,40734,0.62,Sandy,6.169393
1,30629,0.64,Volcanic,5.676648
2,39924,0.69,Volcanic,5.331993



=== farm_management_features ===


,Field_ID,Pollution_level,Plot_size,Crop_type,Annual_yield,Standard_yield
0,40734,0.085267,1.3,0.751354,cassava,0.577964
1,30629,0.399684,2.2,1.069865,cassava,0.486302
2,39924,0.358029,3.4,2.208801,tea,0.649647


## 3) Building the master dataset (SQL join)

We join the four tables on the shared key `Field_ID` to produce a single master table for analysis.

### Why explicit columns (instead of `SELECT *`)?
- Avoids repeated key columns after joins (cleaner schema)
- Makes the master table structure intentional and easier to review


In [19]:
sql_query = """
SELECT
  g.Field_ID,
  g.Elevation, g.Latitude, g.Longitude, g.Location, g.Slope,
  w.Rainfall, w.Min_temperature_C, w.Max_temperature_C, w.Ave_temps,
  s.Soil_fertility, s.Soil_type, s.pH,
  f.Pollution_level, f.Plot_size, f.Crop_type, f.Annual_yield, f.Standard_yield
FROM geographic_features g
JOIN weather_features w ON g.Field_ID = w.Field_ID
JOIN soil_and_crop_features s ON g.Field_ID = s.Field_ID
JOIN farm_management_features f ON g.Field_ID = f.Field_ID
"""

df = pd.read_sql(sql_query, engine)
df.head()


,Field_ID,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Crop_type,Annual_yield,Standard_yield
0,40734,786.05580,-7.389911,-7.556202,Rural_Akatsi,14.795113,1125.2,-3.1,33.1,15.00,0.62,Sandy,6.169393,0.085267,1.3,0.751354,cassava,0.577964
1,30629,674.33410,-7.736849,-1.051539,Rural_Sokoto,11.374611,1450.7,-3.9,30.6,13.35,0.64,Volcanic,5.676648,0.399684,2.2,1.069865,cassava,0.486302
2,39924,826.53390,-9.926616,0.115156,Rural_Sokoto,11.339692,2208.9,-1.8,28.4,13.30,0.69,Volcanic,5.331993,0.358029,3.4,2.208801,tea,0.649647
3,5754,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.277635,cassava,0.532348
4,14146,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,0.043190,1.5,0.832614,wheat,0.555076






## 4) Validation of the joined data (data quality checks)

Before cleaning or analysis, we validate the join result to catch common issues.

### Checks performed
1. **Row/column counts** — quick sanity check that the dataset loaded
2. **Duplicate `Field_ID` rows** — should be 0 if the join key is unique per table
3. **Missing values** — identify completeness issues early
4. **Basic numeric ranges** — spot obvious anomalies (e.g., impossible pH or rainfall)
5. **Simple rule checks** — quick anomaly flags for obviously invalid values


In [20]:
print("Rows:", len(df))
print("Columns:", df.shape[1])

if "Field_ID" not in df.columns:
    raise ValueError("Expected 'Field_ID' in joined dataset, but it was not found.")

print("Duplicate Field_ID rows:", df.duplicated(subset=["Field_ID"]).sum())


Rows: 5654
Columns: 18
Duplicate Field_ID rows: 0


In [22]:
na = df.isna().sum().sort_values(ascending=False)
na.head(15).to_frame("missing_values")


,missing_values
Field_ID,0
Elevation,0
Latitude,0
Longitude,0
Location,0
Slope,0
Rainfall,0
Min_temperature_C,0
Max_temperature_C,0
Ave_temps,0


In [21]:
numeric_candidates = [
    "Rainfall","Min_temperature_C","Max_temperature_C","Ave_temps",
    "pH","Plot_size","Standard_yield","Annual_yield",
    "Soil_fertility","Slope","Elevation","Pollution_level"
]
present = [c for c in numeric_candidates if c in df.columns]
df[present].describe()


,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,pH,Plot_size,Standard_yield,Soil_fertility,Slope,Elevation,Pollution_level
count,5654.000000,5654.000000,5654.000000,5654.000000,5654.000000,5654.000000,5654.000000,5654.000000,5654.000000,5654.000000,5.654000e+03
mean,1201.943244,-4.411956,30.857057,13.222550,5.607073,3.923965,0.534405,0.619308,11.742669,625.468485,2.231923e-01
std,499.782824,1.534330,1.884682,0.801636,0.789215,3.463855,0.111806,0.044791,9.540599,213.768160,2.237976e-01
min,103.100000,-9.700000,24.500000,10.250000,3.700337,0.500000,0.170717,0.510000,0.097316,-878.860800,1.013245e-10
25%,773.175000,-5.500000,29.600000,12.700000,5.030630,1.500000,0.460386,0.590000,4.860120,520.125410,5.133734e-02
50%,1200.600000,-4.300000,30.800000,13.200000,5.602896,2.800000,0.528836,0.620000,9.104626,661.308975,1.647056e-01
75%,1625.425000,-3.300000,32.100000,13.750000,6.162058,4.700000,0.599942,0.650000,15.953242,763.366200,3.058885e-01
max,2470.900000,-0.100000,37.800000,17.300000,7.499959,15.000000,0.898037,0.820000,79.865960,1122.252100,9.954134e-01


In [23]:
invalid_ph_rows = df[(df["pH"] < 0) | (df["pH"] > 14)].shape[0] if "pH" in df.columns else 0
negative_rain_rows = df[df["Rainfall"] < 0].shape[0] if "Rainfall" in df.columns else 0

print("Invalid pH rows (pH < 0 or > 14):", invalid_ph_rows)
print("Negative rainfall rows (Rainfall < 0):", negative_rain_rows)


Invalid pH rows (pH < 0 or > 14): 0
Negative rainfall rows (Rainfall < 0): 0


## 5) Curate the dataset (clean schema + light standardization)

At this step we:
- Standardize a few text columns (trim whitespace)
- Create a `df_clean` that is safe to export and reuse


In [24]:
df_clean = df.copy()

for col in ["Crop_type", "Soil_type", "Location"]:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).str.strip()

df_clean.head()


,Field_ID,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Crop_type,Annual_yield,Standard_yield
0,40734,786.05580,-7.389911,-7.556202,Rural_Akatsi,14.795113,1125.2,-3.1,33.1,15.00,0.62,Sandy,6.169393,0.085267,1.3,0.7513536727525394,cassava,0.577964
1,30629,674.33410,-7.736849,-1.051539,Rural_Sokoto,11.374611,1450.7,-3.9,30.6,13.35,0.64,Volcanic,5.676648,0.399684,2.2,1.0698648105284714,cassava,0.486302
2,39924,826.53390,-9.926616,0.115156,Rural_Sokoto,11.339692,2208.9,-1.8,28.4,13.30,0.69,Volcanic,5.331993,0.358029,3.4,2.2088010770590665,tea,0.649647
3,5754,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.2776351356377693,cassava,0.532348
4,14146,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,0.043190,1.5,0.8326139707528435,wheat,0.555076


## 6) Export the curated dataset

We export the curated dataset so it can be used for downstream EDA/modeling.


In [25]:
out_csv = "smallworld_curated.csv"
df_clean.to_csv(out_csv, index=False)
print("Saved:", out_csv)


Saved: smallworld_curated.csv


## 7) Optional: quick summary check

This section prints a small summary to confirm the curated dataset supports simple analysis (e.g., yield by crop).


In [26]:
from IPython.display import display

if "Crop_type" in df_clean.columns and "Standard_yield" in df_clean.columns:
    summary = (df_clean.groupby("Crop_type")["Standard_yield"]
               .agg(["count","mean","min","max"])
               .sort_values("mean", ascending=False))
    display(summary.head(10))
else:
    print("Expected columns not found. Available columns:", df_clean.columns.tolist())


,count,mean,min,max
Crop_type,,,,
2.065486022397147,1,0.898037,0.898037,0.898037
3.7341768342184034,1,0.889090,0.889090,0.889090
4.256128799779456,1,0.886693,0.886693,0.886693
3.524663999820712,1,0.881166,0.881166,0.881166
4.317585430883226,1,0.881140,0.881140,0.881140
9.765201693516982,1,0.879748,0.879748,0.879748
1.7589309256588699,1,0.879465,0.879465,0.879465
1.4067537665612546,1,0.879221,0.879221,0.879221
0.6115036241269949,1,0.873577,0.873577,0.873577
